In [1]:
from importlib import reload
import platform, os, sys, datetime, re
from os.path import join
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tracker.utils as utils

In [2]:
# default_settings = dict(
#     # Background subtraction.
#     bkgSub_options   = dict( n_training_frames = 200,   # number of frames used to compute background
#                              t_start = 0, t_end = 1200, # time range used to compute background
#                              contrast_factor = 5,       # post-subtraction contrast enhancement factor
#                              secondary_subtraction = True, 
#                              secondary_factor = 2       # frame = frame-secondary_factor*secondary_background
#                            ),
#     t_start          = 0,     # Time at which to start tracking, in seconds.
#     t_end            = 1200,  # Time at which to end tracking, in seconds.
    
#     # Contour detection.
#     n_blur           =  7,    # square-root of n-pixels for threshold blurring
#     block_size       = 15,    # contour block size
#     threshold_offset = 13,    # threshold offset for contour-finding
#     min_area         = 25,    # minimum area for detection
#     max_area         = 600,   # maximum area for detection
#     ideal_area       = 100,   # ideal area to rank contours in first frame (default=(min_area+max_area)/2)
#     max_aspect       = 15,    # maximum aspect ratio for detection
#     ideal_aspect     = 3,     # ideal aspect ratio to rank contours in first frame (default=max_aspect/2)
#     area_penalty     = 0.5,   # weight of area change when connecting fish across frames
#     n_extra          = 2,     # number of extra contours to keep track of
#     morph_transform  = [], 
# #     morph_transform  = [(cv2.MORPH_OPEN,2),(cv2.MORPH_CLOSE,2)], 
# #     morph_transform  = [(cv2.MORPH_CLOSE,20),(cv2.MORPH_OPEN,3)], 
# #     morph_transform  = [(cv2.MORPH_DILATE,5),(cv2.MORPH_OPEN,8)], 
#                               # sequence of morphological operations to perform
#                               # on thresholded image before extracting contours.
#     reversal_threshold = 0.5, # average frame-to-frame displacement against the director 
#                               # over the last few frames to trigger a reversal. 
    
#     # Visualization.
#     save_video       = False, 
#     # What information to draw on the tracking output video.
#     video_output_options = dict( tank=True, points=False, directors=True, 
#                                  extra_points=True, timestamp=True, 
#                                  contours=True, contour_color=(100,255,0), 
#                                  contour_thickness=1 )
#     )

# video_file = '../raw_videos/Pa_Fri_7dpf_GroupD_n2_20200605_1400.avi'
# output_dir = './output/bkgSub'

# Compute the background

### Using a naive average

Compute the background by averaging frames over the entire video. Save it as `background.npy` in the output directory. Use the pre-existing file if there is one.

In [30]:
from cv2.bgsegm import *

# def subtract_background(frame, bkg, bkg_contrast_factor):
#     return 255-np.minimum(255,bkg_contrast_factor*np.absolute(frame-bkg)).astype(np.uint8)    

bg_opt = dict( n_training_frames = 200,   # number of frames used to compute background
               t_start = 0, t_end = 1200, # time range used to compute background
               contrast_factor = 5,       # post-subtraction contrast enhancement factor
               secondary_subtraction = True, 
               secondary_factor = 2       # frame = frame-secondary_factor*secondary_background
             )

class NaiveBackground():
    
    def __init__(self, opt=bg_opt):
        self.name = 'naive'
        self.__dict__.update(bg_opt)
        self.count = 0
    
    def add_training_image(self, img):
        if not hasattr(self,'bg'):
            self.bg  = np.zeros(img.shape,dtype=np.float32)
            self.f32 = np.empty_like(self.bg)
        self.bg    += img
        self.count += 1
        
    def wrapup_training(self):
        self.bg /= self.count
        
    def apply_background(self, img):
#         return img-self.bg
        np.subtract(img, self.bg, out=self.f32)
        np.multiply(self.f32, self.contrast_factor, out=self.f32)
        np.absolute(self.f32, out=self.f32)
        np.minimum(self.f32, 255, out=self.f32)
        img[...] = self.f32
        return img
    
class CvBackground():
    
    def __init__(self,bg_type,n_frames):
        self.name = bg_type
        if bg_type in ['MOG2','KNN','MOG']:
            bg_sub = globals()['createBackgroundSubtractor'+bg_type](history=n_frames)
        elif bg_type in ['CNT','GMG','GSOC','LSBP']:
            bg_sub = globals()['createBackgroundSubtractor'+bg_type]()
        else:
            raise Exception('Unkown background subtractor.')
    
    def add_training_image(self, img):
        pass
    
    def wrapup_training(self):
        # Tell subtractor to stop learning.
        pass
        
    def apply_background(self, img):
        return self.bg.apply(img)

def compute_background(video_file,bg):
    cap         = cv2.VideoCapture(video_file)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_skip  = frame_count//bg.n_training_frames
    for n in range(0,frame_count,frame_skip):
        cap.set(cv2.CAP_PROP_POS_FRAMES,n)
        ret,frame = cap.read()
        bg.add_training_image(frame)
    bg.wrapup_training()
    cap.release()
    return

def make_subtracted_video(video_file,bg):
    cap         = cv2.VideoCapture(video_file)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps         = int(cap.get(cv2.CAP_PROP_FPS))
    width       = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_file = join(output_dir,bg.name+'.mp4')
    fourcc      = cv2.VideoWriter_fourcc(*'mp4v')
    out         = cv2.VideoWriter( filename = output_file, frameSize = (width,height), 
                                   fourcc = fourcc, fps = fps, isColor = True )
    skip        = frame_count//4
    for n in range(0,frame_count-skip,skip):
        cap.set(cv2.CAP_PROP_POS_FRAMES,n)
        for i in range(2*fps):
            ret,frame = cap.read()
            bg.apply_background(frame)
            out.write(255-frame)
    cap.release()
    out.release()
    return

In [32]:
video_file = '../raw_videos/Pa_Fri_7dpf_GroupD_n2_20200605_1400.avi'
output_dir = './output/bkgSub'

bg_opt = dict( n_training_frames = 10,    # number of frames used to compute background
               t_start = 0, t_end = 1200, # time range used to compute background
               contrast_factor = 5,       # post-subtraction contrast enhancement factor
               secondary_subtraction = True, 
               secondary_factor = 2       # frame = frame-secondary_factor*secondary_background
             )

t0          = datetime.datetime.now()
bg          = NaiveBackground(bg_opt)
compute_background(video_file,bg)
t1          = datetime.datetime.now()
print(t1-t0)
make_subtracted_video(video_file,bg)
t2          = datetime.datetime.now()
print(t2-t1)
sys.stdout.flush()

# np.savez_compressed(join(output_dir,'naive.npz'),bg=bg.bg)
# cv2.imwrite(join(output_dir,'naive.png'),bg.bg)



0:00:01.917877
0:00:15.108209


### Using various methods from openCV

In [6]:
globals().update(settings)

from cv2 import createBackgroundSubtractorKNN, createBackgroundSubtractorMOG2
from cv2.bgsegm import *

In [ ]:
for bg in bg_types:
#     help(createBackgroundSubtractorCNT)
    help(globals()['createBackgroundSubtractor'+bg])

In [10]:
Ntraining = 50

bg_types = ['MOG2','KNN','CNT','GMG','GSOC','LSBP','MOG']
# bg_types = ['MOG2','GSOC','CNT']

# bg_sub = cv2.createBackgroundSubtractorMOG2(history=Ntraining) #, varThreshold=25, detectShadows=False)
# bg_sub = createBackgroundSubtractorKNN() #history=10*Ntraining)
# bg_sub = createBackgroundSubtractorCNT()
# bg_sub = createBackgroundSubtractorGMG()
# bg_sub = createBackgroundSubtractorGSOC()
# bg_sub = createBackgroundSubtractorLSBP()
# bg_sub = createBackgroundSubtractorMOG()

bgs_dir = join(output_dir,'backgrounds')
if not os.path.exists(bgs_dir):
    os.mkdir(bgs_dir)

for bg in bg_types:
    
    print(bg)

    cap = cv2.VideoCapture(new_input_file)
    Nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    training_frames = np.linspace(0, Nframes-1, Ntraining, dtype=int)
    
    try:
        opt = dict(history=Ntraining) if bg in ['MOG2','KNN','MOG'] else {}
        bg_sub = globals()['createBackgroundSubtractor'+bg](**opt)

        for i in training_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret,frame = cap.read()
    #         print(ret)
            if ret:
                bg_sub.apply(frame,learningRate=1/Ntraining)
        #         cv2.imwrite(f'test/MOG/{i}.png',bg_sub.getBackgroundImage())

        cap.release()
#         cv2.imwrite(f'test/MOG/{bg}-{Ntraining}.png',bg_sub.getBackgroundImage())
        cv2.imwrite(join(bgs_dir,f'{bg}-{Ntraining}.png'),bg_sub.getBackgroundImage())
    except:
        cap.release()
        %tb

MOG2
KNN
CNT
GMG


error: OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


GSOC


KeyboardInterrupt: 

LSBP


KeyboardInterrupt: 

MOG


KeyboardInterrupt: 